# Init

In [ ]:
import glob
import json
import pandas as pd

def load_and_merge_json_files(directory, filename_pattern):
    # Pfad zum Verzeichnis mit den JSON-Dateien
    path_pattern = f"{directory}/{filename_pattern}*.json"
    data_list = []  # Liste für das Zusammenführen der Daten aus allen Dateien

    # Iterieren über alle Dateien, die dem Muster entsprechen
    for filename in glob.glob(path_pattern):
        with open(filename, 'r') as file:
            try:
                data = json.load(file)
                data_list.extend(data)  # Daten zur Liste hinzufügen
            except:
                print("Error at loading {} file".format(filename))

    return data_list

# Pfad zum Verzeichnis und Dateinamenmuster
directory = "app/Data"
filename_pattern = "received_data"
# Daten aus Dateien laden und zusammenführen
merged_data = load_and_merge_json_files(directory, filename_pattern)

# DataFrame aus der zusammengestellten Liste erstellen
df = pd.DataFrame(merged_data)
df.columns

import app.read_data as rd # Read Data
import numpy as np
import ipaddress
import pandas as pd
from sklearn.cluster import DBSCAN
import nltk


# IPs auf Feature löschen
def __checkValidIP(ip):
    if ip.startswith("http://") or ip.startswith("https://"):
        ip = ip.split("//")[1].split(":")[0]
    else:
        ip = ip.split(":")[0]
    ip = ip.split("/")[0]
    if "localhost" in ip:
        return True
    try:
        ipaddress.ip_address(ip)
        return True
    except ValueError:
        return False

def __remove_ip(x):
    x = str(x)
    if __checkValidIP(x):
        return ""
    else:
        return x
    
def __remove_ip_features(df):
    features_list = ["Host", "Referer", "X-Forwarded-For", "Origin"]
    headers = df["Headers"].apply(pd.Series)
    for feature in features_list:
        headers[feature] = headers[feature].apply(__remove_ip)
    df = df.drop(columns="Headers")
    df = pd.concat([df,headers],axis=1)
    return df

df = __remove_ip_features(df)



# NaN Values ersetzten
def fill_nan(df):
    for column in df.columns:
        df[column].fillna("", inplace=True)
        df[column].replace("NaN","",inplace=True)
        df[column].replace("nan","",inplace=True)
    return df

df = fill_nan(df)



# Original DF speichern
df_origin = df



# Entfernen von Timestamp, SourceIP und SourcePort
def __remove_features(df):
    selected_features = [col for col in df.columns if col not in ['Timestamp', 'SourceIP', 'SourcePort']]
    return df[selected_features].copy()

df = __remove_features(df)



# Markieren von Duplikaten und Entfernen davon
df["Duplicate"] = df.duplicated(keep=False)
df.drop_duplicates(keep='first',inplace=True,ignore_index=True)



# Leere Spalten löschen
df = df.dropna(axis=1, how='all')

def delete_cluster_columns(df_temp):
    columns_to_drop = [col for col in df_temp.columns if col.startswith("Ex_")]
    df_temp.drop(columns=columns_to_drop, inplace=True)
    return df_temp

def concatenate_rows(df):
    def concatenate_row(row):
        return ''.join(row.astype(str))
    arr_string_full_concat = df.apply(concatenate_row, axis=1).values
    return arr_string_full_concat

dataset_1 = concatenate_rows(df)

def concatenate_equalize_length(df):
    def concatenate_row(row, max_lengths):
        equalized_row = [str(val).ljust(max_lengths.iloc[idx]) for idx, val in enumerate(row)]
        return ''.join(equalized_row)

    max_lengths = df.apply(lambda x: x.astype(str).str.len()).max()
    concatenated_array = df.apply(lambda row: concatenate_row(row, max_lengths), axis=1).values
    return concatenated_array

dataset_2 = concatenate_equalize_length(df)

dataset_3 = df.drop(columns=["Duplicate"])
dataset_3 = delete_cluster_columns(dataset_3)

dataset_4 = df.drop(columns=["Duplicate", "User-Agent"])
dataset_4 = delete_cluster_columns(dataset_4)

dataset_5 = df.drop(columns=["Duplicate", "User-Agent"])
dataset_5 = delete_cluster_columns(dataset_5)
for column in dataset_5.columns:
    counts = dataset_5[column].value_counts()
    frequent_entries = counts[counts >= 20].index.tolist()
    dataset_5.loc[dataset_5[column].isin(frequent_entries), column] = ""
    dataset_5 = dataset_5.dropna(axis=1, how='all')

dataset_6 = dataset_4
dataset_6['Path'] = dataset_6['Path'].str[1:]
remove_extensions = [".php", ".xml", ".html", "index", ".js", ".ico", ".txt", ".env"]
for substring in remove_extensions:
    dataset_6['Path'] = dataset_6['Path'].str.replace(substring, '', regex=False)

import string
from sklearn.feature_extraction.text import CountVectorizer

def bag_of_chars(str):
    # Combine all printable ASCII characters and digits
    chars = string.ascii_letters + string.digits + string.punctuation

    # Create a bag-of-chars for each string
    vectorizer = CountVectorizer(analyzer='char', lowercase=False, vocabulary=list(chars))
    bag = vectorizer.fit_transform([str]).toarray().flatten()
    return bag

def normalized_levenshtein_distance(s1, s2):
    # Berechnung der Levenshtein-Distanz
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            dp[i][j] = min(dp[i-1][j] + 1,
                           dp[i][j-1] + 1,
                           dp[i-1][j-1] + cost)
    
    levenshtein_distance = dp[m][n]
    # Maximale mögliche Distanz ist die Länge des längeren Strings
    max_distance = max(m, n)
    # Normalisierung der Levenshtein-Distanz
    normalized_distance = 1 - levenshtein_distance / max_distance
    return normalized_distance

def array_similarity(array1, array2):
    # Anzahl der nicht vorhandenen Zeichen in beiden Arrays zählen
    non_matching_chars = sum(1 for a1, a2 in zip(array1, array2) if a1 == 0 and a2 == 0)
    # Anzahl der übereinstimmenden Zeichen zählen
    matching_chars = sum(min(a1, a2) for a1, a2 in zip(array1, array2))
    # Gesamtsumme der Zeichen in beiden Arrays zählen
    total_chars = sum(array1) + sum(array2)
    # Übereinstimmung in Prozent berechnen
    # Bei zwei leeren Strings
    if total_chars == 0:
        return 1
    return matching_chars / (abs(total_chars)/2)

def bag_of_chars_and_selfdesigned_similarity(str1, str2):
    bag1 = bag_of_chars(str1)
    bag2 = bag_of_chars(str2)

    # Calculate the cosine similarity
    return array_similarity(bag1, bag2)
    



In [2]:
from sklearn.cluster import OPTICS
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np

def calculate_silhouette_coefficient(distance_matrix, labels):
    return silhouette_score(distance_matrix, labels, metric="precomputed")

def optics_clustering(dm_ex, min_samples, xi, eps):
    optics = OPTICS(min_samples=min_samples, xi=xi, max_eps=eps)
    # Fit the model to the data
    optics.fit(dm_ex)
    # Return the clustering labels
    return optics.labels_

def find_optimal_parameters_optics(data, min_samples_range=[2], xi_range=[0.01], eps_range=[2]):
    optimal_params = {'min_samples': min_samples_range[0], 'xi': xi_range[0],'eps': eps_range[0]}
    max_silhouette = -1
    best_labels = None
    for min_samples in min_samples_range:
        for xi in xi_range:
            for eps in eps_range:
                print("Parameters - Xi: {} Eps: {}".format(xi,eps))
                optics = OPTICS(min_samples=min_samples, xi=xi,max_eps=eps).fit(data)
                labels = optics.labels_
                n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
                
                if 1 < n_clusters < len(data) - 1:
                    silhouette = silhouette_score(data, labels)
                    if silhouette > max_silhouette:
                        max_silhouette = silhouette
                        optimal_params['min_samples'] = min_samples
                        optimal_params['xi'] = xi
                        optimal_params['eps'] = eps
                        best_labels = labels
    return optimal_params, max_silhouette, best_labels, optics

# Outlier Detection

## Prefiltering mit Clustering

In [ ]:
import warnings
warnings.filterwarnings('ignore')


def add_findings_df_prefiltered(results):
    outlier_ind = dict()
    for column in results:
        matching_rows = results[results[column] == True]
        indices = matching_rows.index
        outlier_ind[column] = indices
    for column in outlier_ind:
        for idx in outlier_ind[column]:
            if df_prefiltered.at[idx,"Outlier"] is None:
                df_prefiltered.at[idx,"Outlier"] = [column]
            elif column not in df_prefiltered.at[idx,"Outlier"]:
                df_prefiltered.at[idx,"Outlier"].append(column)

def find_long_values(df, whitelist_Features, threshold):
    results = pd.DataFrame(index=df.index)
    # Iterate over each column in the DataFrame
    for col in df.columns:
        if col not in whitelist_Features:
            # Calculate the threshold length for this column
            col_len = df[col].apply(len)
            threshold_len = col_len.quantile(threshold)
            exceptionally_long = col_len > threshold_len
            results[f'{col}'] = exceptionally_long

    add_findings_df_prefiltered(results)
    return results

import re

def find_special_chars(df, whitelist_Features=[], threshold=0.1):
    results = pd.DataFrame(index=df.index)
    # Iterate over each column in the DataFrame
    for col in df.columns:
        if col not in whitelist_Features:
            # Calculate the threshold length for this column
            col_values = df[col]
            col_values = col_values[col_values.str.len() > 5]
            special_chars = re.compile(r'[^\w\s]')
            num_special_chars = col_values.apply(lambda x: len(special_chars.findall(x)))
            threshold_num_special_chars = num_special_chars.quantile(threshold)
            exceptionally_high_special_chars = num_special_chars > threshold_num_special_chars
            results[f'{col}'] = exceptionally_high_special_chars
    add_findings_df_prefiltered(results)
    return results

def find_spaces(df, whitelist_Features=[], threshold=0.5):
    results = pd.DataFrame(index=df.index)
    # Iterate over each column in the DataFrame
    for col in df.columns:
        if col not in whitelist_Features:
            # Calculate the threshold length for this column
            col_values = df[col]
            col_values = col_values[col_values.str.len() > 5]
            num_spaces = col_values.apply(lambda x: x.count(' '))
            threshold_num_spaces = num_spaces.quantile(threshold)
            exceptionally_high_spaces = num_spaces > threshold_num_spaces
            results[f'{col}'] = exceptionally_high_spaces
    add_findings_df_prefiltered(results)
    return results

def find_keywords(df, keywords=['cd', 'curl', 'rm ', 'cmd', 'wget', 'rm%20', 'Base64', 'shell']):
    results = pd.DataFrame(index=df.index)
    # Iterate over each column in the DataFrame
    for col in df.columns:
            has_keyword = df[col].apply(lambda x: any(keyword in x for keyword in keywords))
            results[f'{col}'] = has_keyword
    add_findings_df_prefiltered(results)
    return results

def remove_duplicates_from_outlier(df_temp):
    # Check if 'Duplicate' and 'Outlier' columns exist in the DataFrame
    if 'Duplicate' not in df_temp.columns or 'Outlier' not in df_temp.columns:
        raise ValueError("DataFrame must contain 'Duplicate' and 'Outlier' columns")

    for index, row in df_temp.iterrows():
        if row['Duplicate'] == True:
            df_temp.at[index, 'Outlier'] = []
    return df_temp

df_prefiltered = df.copy()
df_prefiltered["Outlier"] = None
# Prefiltering
whitelist_Features = ["Path"]
results = find_long_values(dataset_6,whitelist_Features, 0.995)
results = find_special_chars(dataset_6,threshold=0.95)
results = find_spaces(dataset_6,threshold=0.90)
results = find_keywords(dataset_6)

df_prefiltered = remove_duplicates_from_outlier(df_prefiltered)

# Assuming df is your DataFrame and "Outlier" is the column of interest
exploded_df = df_prefiltered['Outlier'].explode().dropna().reset_index(drop=True)
unique_values = exploded_df.unique()

dfs = {}
for col in unique_values:
    df_prefiltered['Outlier'] = df_prefiltered['Outlier'].apply(lambda x: x if x is not None else [])
    dfs[col] = df_prefiltered[df_prefiltered['Outlier'].apply(lambda x: col in x)]
from sklearn.metrics import silhouette_score
def distance_matrix_lev(array):
    # Get the length of the array
    n = len(array)
    # Initialize the distance matrix as an empty matrix
    dist_matrix = np.zeros((n, n))
    # Get the unique values in the array
    unique_values = np.unique(array)
    # Compute the cosine similarity between each pair of strings in the unique values
    for i, value1 in enumerate(unique_values):
        for j, value2 in enumerate(unique_values):
            if i <= j:
                if value1 != value2:
                            # Calculate the cosine similarity
                    if len(value1) == 0 ^ len(value2) == 0:
                        similarity = 0
                    if len(value1) == 0 and len(value2) == 0:
                        similarity = 1
                    else:
                        similarity = normalized_levenshtein_distance(value1,value2)
                    # Get the indices of value1 and value2 in the input array
                    idx1 = np.where(array == value1)[0]
                    idx2 = np.where(array == value2)[0]
                    # Assign the calculated similarity to the correct positions in the distance matrix
                    for n in idx1:
                        for m in idx2:
                            dist_matrix[n, m] = 1 - similarity
                            dist_matrix[m, n] = 1 - similarity
    return dist_matrix


def process_column_data_prefiltering(column_name,data):
    ex_num = f"Cluster_Prefiltering_{column_name}"
    if len(data) > 1:
        dm_ex = distance_matrix_lev(data)
        eps, max_sil, labels = find_optimal_parameters_optics(dm_ex)
        print("For Category {} Requests: {}  the best eps: {} with sil-score: {}".format(column_name,len(dfs[column_name]), eps, max_sil))
        dfs[column_name].loc[:,ex_num] = labels  # Speichere die Labels im entsprechenden DataFrame
    else:
        dfs[column_name].loc[:,ex_num] = -1



for col in unique_values:
    data = dfs[col][col]
    process_column_data_prefiltering(col,data)

ex_num = "Cluster_Prefiltering"

for col in unique_values:
    new_col_name = "{}_{}".format(ex_num, col)
    if new_col_name not in df_prefiltered.columns:
        df_prefiltered[new_col_name] = np.nan  # Verwenden Sie np.nan für fehlende Daten
    df_prefiltered.loc[dfs[col].index, new_col_name] = dfs[col][new_col_name]
# Angenommen, fill_nan ist eine Funktion, die NaN-Werte behandelt
df_prefiltered = fill_nan(df_prefiltered)

def analyse_prefiltered_outlier(df_prefiltered):
    # Initialisierung der Spalte 'Outlier_Detected' mit False
    df_prefiltered['Outlier_Detected'] = None
    # Initialisiere 'Outlier_Category' als None, um später Arrays zu speichern
    df_prefiltered['Outlier_Category'] = None

    for col in df_prefiltered.columns[df_prefiltered.columns.str.startswith('Cluster_')]:
        # Finde die Indizes der Ausreißer (Werte von -1) in der aktuellen Spalte
        outlier_indices = df_prefiltered[df_prefiltered[col] == -1].index
        non_outlier_indices = df_prefiltered[df_prefiltered[col] != ""].index
        for idx in non_outlier_indices:
            if df_prefiltered.at[idx, 'Outlier_Detected'] != True:
                df_prefiltered.at[idx, 'Outlier_Detected'] = False

        for idx in outlier_indices:
            # Markiere die Zeile als Outlier
            df_prefiltered.at[idx, 'Outlier_Detected'] = True

            # Wenn 'Outlier_Category' noch nicht initialisiert wurde, füge die erste Spalte hinzu
            if df_prefiltered.at[idx, 'Outlier_Category'] is None:
                df_prefiltered.at[idx, 'Outlier_Category'] = [col.removeprefix(ex_num+"_")]
            # Wenn die Spalte noch nicht in 'Outlier_Category' enthalten ist, füge sie hinzu
            elif col not in df_prefiltered.at[idx, 'Outlier_Category']:
                df_prefiltered.at[idx, 'Outlier_Category'].append(col.removeprefix(ex_num+"_"))
    return df_prefiltered

df_prefiltered = analyse_prefiltered_outlier(df_prefiltered)
df_prefiltered = fill_nan(df_prefiltered)
df['Outlier_Detected'] = df_prefiltered['Outlier_Detected']
df['Outlier_Category'] = df_prefiltered['Outlier_Category']

### Temp Output

In [25]:
df_temp = dataset_6
for column in df_temp:
    unique_values = df_temp[column].value_counts()
    file_name = f"output_{column}.txt"
    with open(file_name, 'w') as file:
        for value, count in unique_values.items():
            file.write(f"{value}: {count}\n")

## Distanzmatritzen und Clustering für jedes Feature durchführen

In [ ]:
from sklearn.metrics import silhouette_score
def distance_matrix_self(array):
    # Get the length of the array
    n = len(array)
    # Initialize the distance matrix as an empty matrix
    dist_matrix = np.zeros((n, n))
    # Get the unique values in the array
    unique_values = np.unique(array)
    # Compute the cosine similarity between each pair of strings in the unique values
    for i, value1 in enumerate(unique_values):
        for j, value2 in enumerate(unique_values):
            if i <= j:
                if value1 != value2:
                            # Calculate the cosine similarity
                    if len(value1) == 0 ^ len(value2) == 0:
                        similarity = 0
                    if len(value1) == 0 and len(value2) == 0:
                        similarity = 1
                    else:
                        similarity = bag_of_chars_and_selfdesigned_similarity(value1,value2)
                    # Get the indices of value1 and value2 in the input array
                    idx1 = np.where(array == value1)[0]
                    idx2 = np.where(array == value2)[0]
                    # Assign the calculated similarity to the correct positions in the distance matrix
                    for n in idx1:
                        for m in idx2:
                            dist_matrix[n, m] = 1 - similarity
                            dist_matrix[m, n] = 1 - similarity
    return dist_matrix

df_features = {}

def process_column_data_features(column_name,data):
    ex_num = f"Cluster_Features_{column_name}"

    if len(data) > 1:
        print("Calc Distancematrix")
        dm_ex = distance_matrix_self(data)
        print("Clustering...")
        eps, max_sil, labels, optics = find_optimal_parameters_optics(dm_ex)
        print("For Category {} Requests: {}  the best eps: {} with sil-score: {}".format(column_name,len(df[column_name]), eps, max_sil))
        outlier_indices = np.where(labels == -1)[0]
        for idx in outlier_indices:
            df.at[idx,'Outlier_Detected'] = True  
            if len(df.at[idx, 'Outlier_Category']) == 0:
                df.at[idx, 'Outlier_Category'] = [column_name]
            elif column_name not in df.at[idx, 'Outlier_Category']:
                df.at[idx, 'Outlier_Category'].append(column_name)
    else:
        print("{} skipped due to length 1".format(column_name))
    return optics


unique_values = dataset_6.columns

for col in unique_values:
    df_features[col] = None

for col in unique_values:
        print("Calculate for Feature: {}".format(col))
        data = dataset_6[col]
        optics = process_column_data_features(col,data)


#1143 Minuten